In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
!unzip -o "/content/drive/My Drive/HACKATHON/annotations_virat.zip" -d /content/

Archive:  /content/drive/My Drive/HACKATHON/annotations_virat.zip
  inflating: /content/annotations_virat/classes.txt  
  inflating: /content/annotations_virat/video10_frame1.jpg  
  inflating: /content/annotations_virat/video10_frame1.txt  
  inflating: /content/annotations_virat/video10_frame2.jpg  
  inflating: /content/annotations_virat/video10_frame2.txt  
  inflating: /content/annotations_virat/video10_frame3.jpg  
  inflating: /content/annotations_virat/video10_frame3.txt  
  inflating: /content/annotations_virat/video10_frame4.jpg  
  inflating: /content/annotations_virat/video10_frame4.txt  
  inflating: /content/annotations_virat/video10_frame5.jpg  
  inflating: /content/annotations_virat/video10_frame5.txt  
  inflating: /content/annotations_virat/video11_frame1.jpg  
  inflating: /content/annotations_virat/video11_frame1.txt  
  inflating: /content/annotations_virat/video11_frame2.jpg  
  inflating: /content/annotations_virat/video11_frame2.txt  
  inflating: /content/anno

In [72]:
# Clone YOLOv5 repo
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17516, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 17516 (delta 7), reused 0 (delta 0), pack-reused 17496 (from 4)
Receiving objects: 100% (17516/17516), 16.66 MiB | 16.67 MiB/s, done.
Resolving deltas: 100% (11998/11998), done.
/content/yolov5/yolov5


In [77]:
import os

for root, dirs, files in os.walk('/content/annotations_virat/', topdown=True):
    print(f"📁 {root}")
    for name in files[:5]:  # Show only 5 files per folder
        print("     └──", name)


📁 /content/annotations_virat/
     └── video2_frame4.jpg
     └── video2_frame3.jpg
     └── video3_frame4.txt
     └── video20_frame4.jpg
     └── video10_frame2.txt


In [78]:
import os
import shutil

src = '/content/annotations_virat'
dst_img = '/content/dataset_virat/images'
dst_lbl = '/content/dataset_virat/labels'

# Create target folders
os.makedirs(dst_img, exist_ok=True)
os.makedirs(dst_lbl, exist_ok=True)

# Get base filenames
all_files = os.listdir(src)
jpgs = {os.path.splitext(f)[0] for f in all_files if f.endswith('.jpg')}
txts = {os.path.splitext(f)[0] for f in all_files if f.endswith('.txt')}

# Only those that have both .jpg and .txt
valid = jpgs & txts

# Copy to cleaned folder
for base in valid:
    shutil.copy(os.path.join(src, base + '.jpg'), os.path.join(dst_img, base + '.jpg'))
    shutil.copy(os.path.join(src, base + '.txt'), os.path.join(dst_lbl, base + '.txt'))

print(f"✅ Filtered {len(valid)} matched image-label pairs.")


✅ Filtered 57 matched image-label pairs.


In [79]:
import os
import shutil
import random

img_dir = '/content/dataset_virat/images'
lbl_dir = '/content/dataset_virat/labels'
output_dir = '/content/virat_yolo_dataset'

train_ratio = 0.8  # 80% train, 20% valid

# Create folders
for split in ['train', 'valid']:
    os.makedirs(f'{output_dir}/{split}/images', exist_ok=True)
    os.makedirs(f'{output_dir}/{split}/labels', exist_ok=True)

# Get all base filenames
base_files = [os.path.splitext(f)[0] for f in os.listdir(img_dir)]
random.shuffle(base_files)
train_cutoff = int(len(base_files) * train_ratio)

train_files = base_files[:train_cutoff]
valid_files = base_files[train_cutoff:]

# Copy files
for split, files in [('train', train_files), ('valid', valid_files)]:
    for f in files:
        shutil.copy(f'{img_dir}/{f}.jpg', f'{output_dir}/{split}/images/{f}.jpg')
        shutil.copy(f'{lbl_dir}/{f}.txt', f'{output_dir}/{split}/labels/{f}.txt')

print(f"✅ Train: {len(train_files)}, Valid: {len(valid_files)}")


✅ Train: 45, Valid: 12


In [81]:
yaml_content = """
train: /content/virat_yolo_dataset/train/images
val: /content/virat_yolo_dataset/valid/images

nc: 16
names: [dog, person, cat, tv, car, meatballs, marinara sauce, tomato soup, chicken noodle soup,
        french onion soup, chicken breast, ribs, pulled pork, hamburger, cavity, restricted area]
"""

with open("/content/virat_yolo_dataset/data.yaml", "w") as f:
    f.write(yaml_content)

print("✅ data.yaml created successfully.")


✅ data.yaml created successfully.


In [82]:
%cd /content/yolov5
!python train.py --img 416 --batch 8 --epochs 20 --data /content/virat_yolo_dataset/data.yaml --weights yolov5s.pt --name yolo_virat_16cls


/content/yolov5
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-07-27 08:16:08.332975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753604168.359691   36519 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753604168.367962   36519 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Currently logged in as: shailvisuman (shailvisuman-iit-madras-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=, data=/content/virat_yolo_dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs

In [84]:
import os

img_dir = "/content/annotations_virat"  # or change path accordingly
all_jpgs = set(f for f in os.listdir(img_dir) if f.endswith(".jpg"))
all_txts = set(f.replace(".txt", ".jpg") for f in os.listdir(img_dir) if f.endswith(".txt"))

unlabeled_jpgs = list(all_jpgs - all_txts)

print(f"Found {len(unlabeled_jpgs)} unlabeled images.")
if unlabeled_jpgs:
    test_image_path = os.path.join(img_dir, unlabeled_jpgs[0])
    print(f"Using image: {test_image_path}")
else:
    print("No unmatched images found.")


Found 68 unlabeled images.
Using image: /content/annotations_virat/video9_frame2.jpg


In [87]:
from IPython.display import Image
Image(filename='runs/detect/exp2/video9_frame1.jpg')


FileNotFoundError: [Errno 2] No such file or directory: 'runs/detect/exp2/video9_frame1.jpg'

In [88]:
!python detect.py --weights runs/train/yolo_virat_16cls/weights/best.pt --img 416 --conf 0.3 --source "/content/annotations_virat/video9_frame1.jpg"


detect: weights=['runs/train/yolo_virat_16cls/weights/best.pt'], source=/content/annotations_virat/video9_frame1.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7053277 parameters, 0 gradients, 15.9 GFLOPs
image 1/1 /content/annotations_virat/video9_frame1.jpg: 416x416 (no detections), 181.0ms
Speed: 1.5ms pre-process, 181.0ms inference, 0.3ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp3
